In [56]:
from custom_eval import eval_video
import os

MAX_BATCHES = 113

def checkpoint_time(checkpoint: str):
    checkpoint = checkpoint.split('-')
    epoch = int(checkpoint[0][5:])
    batch = int(checkpoint[1][5:-3])
    return epoch, batch

def make_schedule(n_batches):
    n = int((n_batches + 1) / 2)
    ret = [0, n]
    n = int((n + 1) / 2)
    i = 1
    end = False
    while True:
        prev = ret[-i:]
        for p in prev:
            if p-n not in ret and p-n > 0 : ret.append(p - n)
            if p+n not in ret and p+n < n_batches: ret.append(p + n)

        if end or n == 0: break
        end = n == 1
        n = int((n+1) / 2)
        i = i * 2
    return list(ret)

def normalize_batch(epoch, batch):
    return epoch + (batch / MAX_BATCHES)

def get_checkpoints(run_name: str):
    weights_dir = os.path.join('runs', 'detect', run_name, 'weights')
    checkpoints = sorted(
        [c for c in os.listdir(weights_dir) if 'batch' in c],
        key=lambda x: normalize_batch(*checkpoint_time(x))
    )
    return weights_dir, checkpoints

In [57]:
for i in range(2, 10):
    print(f"{i}: {make_schedule(i)}")

2: [0, 1]
3: [0, 2, 1]
4: [0, 2, 1, 3]
5: [0, 3, 1, 2, 4]
6: [0, 3, 1, 5, 2, 4]
7: [0, 4, 2, 6, 1, 3, 5]
8: [0, 4, 2, 6, 1, 3, 5, 7]
9: [0, 5, 2, 8, 4, 6, 1, 3, 7]


In [58]:
from ultralytics import YOLO
from custom_eval import eval_video, get_test_files, BINARY_KEYS
import json
TEST_DIR = "datasets/04_Test_Datasets/"

test_files = get_test_files(TEST_DIR)

def eval_checkpoints(run_name):
    weights_dir, checkpoints = get_checkpoints(run_name)
    n_epochs = len(set([checkpoint_time(c)[0] for c in checkpoints]))
    n_batches = len(set([checkpoint_time(c)[1] for c in checkpoints]))
    checkpoints_2d = [
        [c for c in checkpoints if checkpoint_time(c)[0] == e]
        for e in range(n_epochs)
    ]
    schedule = make_schedule(n_batches)
    print(schedule)
    for batch in schedule:
        for epoch in range(n_epochs):
            checkpoint = checkpoints_2d[epoch][batch]
            print(f"EVAL ON EPOCH {epoch}, BATCH {batch} ({checkpoint})")
            model = YOLO(os.path.join(weights_dir, checkpoint))
            res = eval_video(model, test_files, 1e-3, 0.6, BINARY_KEYS)
            real_batch = checkpoint_time(checkpoint)[1]
            res['epoch'] = epoch
            res['batch'] = real_batch
            with open(os.path.join('runs', 'detect', run_name, 'bias.jsonl'), 'a') as f:
                f.write(json.dumps(res) + "\n")



In [59]:
eval_checkpoints('train')

[0, 7, 3, 11, 1, 5, 9, 10, 12, 2, 4, 6, 8]
EVAL ON EPOCH 0, BATCH 0 (epoch0-batch0.pt)
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


100%|██████████| 81/81 [03:05<00:00,  2.30s/it]


NUM CORRECT: 22 / 81 (0.27)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Neg': 4,
          'False_Pos': 52,
          'True_Neg': 21,
          'True_Pos': 4,
          'guess_rate': 0.691358024691358},
 'plane': {'False_Neg': 43,
           'False_Pos': 7,
           'True_Neg': 13,
           'True_Pos': 18,
           'guess_rate': 0.30864197530864196}}
EVAL ON EPOCH 1, BATCH 0 (epoch1-batch0.pt)
starting eval...


...

NUM CORRECT: 52 / 81 (0.64)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Neg': 2,
          'False_Pos': 27,
          'True_Neg': 46,
          'True_Pos': 6,
          'guess_rate': 0.4074074074074074},
 'plane': {'False_Neg': 15,
           'False_Pos': 2,
           'True_Neg': 18,
           'True_Pos': 46,
           'guess_rate': 0.5925925925925926}}
EVAL ON EPOCH 2, BATCH 7 (epoch2-batch70.pt)
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


 25%|██▍       | 20/81 [00:32<01:39,  1.63s/it]


KeyboardInterrupt: 